### Funkcje bo charted nie działa

In [1]:
import geopandas as gpd
from  imgw_static import get_map_zlewnie, get_static_data_hydro,get_static_data_meteo
import pandas as pd
def get_hydro_metadata() -> gpd.GeoDataFrame:
    """
    Wczytuje metadane hydrologiczne ze wskazanego URL (np. API IMGW)
    i zwraca je jako GeoDataFrame w układzie EPSG:2180 (PUWG 1992).
    """
    hydro_url = "http://danepubliczne.imgw.pl/api/data/hydro2/"
    hydro_stat = pd.read_json(hydro_url)
    df = hydro_stat.iloc[:, 0:4]
    
    # Tworzenie geometrii
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["lon"], df["lat"]),
        crs="EPSG:4326"
    )
    gdf.columns = ['Station Code','Station Name','Lon','Lat','geometry']
    # Konwersja do układu PL (PUWG 1992)
    gdf = gdf.to_crs("EPSG:2180")
    
    return gdf
def get_meteo_metadata() -> gpd.GeoDataFrame:
    """
    Wczytuje metadane meteorologiczne ze wskazanego URL (np. API IMGW)
    i zwraca je jako GeoDataFrame w układzie EPSG:2180 (PUWG 1992).
    """
    meteo_url = "http://danepubliczne.imgw.pl/api/data/meteo/"
    meteo_stat = pd.read_json(meteo_url)
    df = meteo_stat.iloc[:, 0:4]
    
    # Tworzenie geometrii
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["lon"], df["lat"]),
        crs="EPSG:4326"
    )
    gdf.columns = ['Station Code','Station Name','Lon','Lat','geometry']
    # Konwersja do układu PL (PUWG 1992)
    gdf = gdf.to_crs("EPSG:2180")
    
    return gdf

# Preprocessing

## Dane Hydrologiczne

In [2]:
import pandas as pd

df = get_static_data_hydro()

Dane Pomiarowe

In [3]:
print("Shape: ", df.shape)
print("Ilość stacji hydro w danych pomiarowych: ",df["Station Code"].nunique())
df.head()


Shape:  (4273340, 10)
Ilość stacji hydro w danych pomiarowych:  917


,Station Code,Station Name,Name,Hydro Year,Hydro Month,Day,Water Level,Flow,Water Temp,Calendar Month
0,149180020,CHA£UPKI,Odra (1),2010,1,1,158,30.8,99.9,11
1,149180020,CHA£UPKI,Odra (1),2010,1,2,154,28.5,99.9,11
2,149180020,CHA£UPKI,Odra (1),2010,1,3,150,26.0,99.9,11
3,149180020,CHA£UPKI,Odra (1),2010,1,4,149,25.4,99.9,11
4,149180020,CHA£UPKI,Odra (1),2010,1,5,149,25.9,99.9,11


Dane Meta

In [4]:
gdf = get_hydro_metadata()
print("Shape: ", gdf.shape)
print("Ilość stacji hydro w meta danych: ",gdf["Station Code"].nunique())
gdf.head()

Shape:  (859, 5)
Ilość stacji hydro w meta danych:  859


,Station Code,Station Name,Lon,Lat,geometry
0,150160330,SZCZYTNA,16.443056,50.415556,POINT (318401.478 286283.506)
1,150160340,SARNY,16.465833,50.547778,POINT (320520.661 300923.221)
2,150160350,SZALEJÓW GÓRNY,16.537222,50.418333,POINT (325098.752 286366.241)
3,150160360,STARKÓW,16.580000,50.377500,POINT (327988.637 281728.229)
4,150160370,TOPOLICE,16.609167,50.366944,POINT (330023.848 280487.958)


## Wstępna selekcja danych hydrologicznych

Wybieramy stacje które:

1. Rozpoczęły pomiary przed lub w 2010 roku.
2. Można zmapować do ich lokalizacji.

In [5]:
print("Ile stacji rozpoczęło pomiary przed lub w 2010 roku:",(df.groupby("Station Code")["Hydro Year"].min() == 2010).mean() * 100,"%")

Ile stacji rozpoczęło pomiary przed lub w 2010 roku: 92.80261723009815 %


Odcinamy stacje z pierszwego warunku.

In [6]:
valid_stations = df.groupby("Station Code")["Hydro Year"].min()
valid_stations = valid_stations[valid_stations == 2010].index

# Step 2: Filter the DataFrame to include only those stations
filtered_df = df[df["Station Code"].isin(valid_stations)]

Odcinamy stacje z drugiego warunku.

In [7]:
valid_stations = gdf['Station Code']

# Step 3: Filter the DataFrame to include only those stations
filtered_df_2 = filtered_df[filtered_df["Station Code"].isin(valid_stations)]

In [8]:
print("Shape: ", filtered_df_2.shape)
print("Ilość stacji w odfiltrowanych hydro danych: ",filtered_df_2["Station Code"].nunique())

Shape:  (3533386, 10)
Ilość stacji w odfiltrowanych hydro danych:  697


Eksportujemy odfiltrowane dane do static_data

W efekcie mamy spójne dane zaczynające się od 2010 i z lokalizacją

In [11]:
filtered_df_2.to_parquet("../static_data/filtered_hydro_data.parquet.gzip")

In [15]:
filtered_df_2

,Station Code,Station Name,Name,Hydro Year,Hydro Month,Day,Water Level,Flow,Water Temp,Calendar Month
0,149180020,CHA£UPKI,Odra (1),2010,1,1,158,30.80,99.9,11
1,149180020,CHA£UPKI,Odra (1),2010,1,2,154,28.50,99.9,11
2,149180020,CHA£UPKI,Odra (1),2010,1,3,150,26.00,99.9,11
3,149180020,CHA£UPKI,Odra (1),2010,1,4,149,25.40,99.9,11
4,149180020,CHA£UPKI,Odra (1),2010,1,5,149,25.90,99.9,11
...,...,...,...,...,...,...,...,...,...,...
301989,149190250,JABŁONKA,Piekielnik (82224),2023,12,27,155,2.32,99.9,10
301990,149190250,JABŁONKA,Piekielnik (82224),2023,12,28,174,5.64,99.9,10
301991,149190250,JABŁONKA,Piekielnik (82224),2023,12,29,158,2.87,99.9,10
301992,149190250,JABŁONKA,Piekielnik (82224),2023,12,30,150,1.48,99.9,10


### Dane Meteo

Czekam na dostarczenie danych czysto meteo, zamiast synoptycznych.